In [208]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import selenium
import re
import nltk

In [209]:
from selenium import webdriver
from selenium.webdriver import Firefox
browser=Firefox()
url="https://arxiv.org/"

In [210]:
browser.get(url)

In [211]:
from selenium.webdriver.support.ui import Select
from selenium import webdriver

In [212]:
browser.find_element_by_class_name("toggle-control").click()


In [213]:
browser.find_element_by_xpath("//div[@class='field has-addons']//input[@class='input' and starts-with(@placeholder, 'Search...')]").send_keys("healthcare")

In [214]:
browser.find_element_by_xpath("//div[@class='field has-addons']//button[@class='button']").click()

In [215]:
from bs4 import BeautifulSoup
html=browser.page_source
html
soup=BeautifulSoup(html,'html.parser')
soup

<html lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- new favicon config and versions by realfavicongenerator.net -->
<link href="https://static.arxiv.org/static/base/0.16.8/images/icons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://static.arxiv.org/static/base/0.16.8/images/icons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://static.arxiv.org/static/base/0.16.8/images/icons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="https://static.arxiv.org/static/base/0.16.8/images/icons/site.webmanifest" rel="manifest"/>
<link color="#b31b1b" href="https://static.arxiv.org/static/base/0.16.8/images/icons/safari-pinned-tab.svg" rel="mask-icon"/>
<link href="https://static.arxiv.org/static/base/0.16.8/images/icons/favicon.ico" rel="shortcut icon"/>
<meta content="#b31b1b" name="msapplication-TileColor"/>
<meta content="images/icons

In [216]:
for link in soup.find_all('span' , class_ = "abstract-full has-text-grey-dark mathjax"):
    print("Abtract: {}" .format(link.text))

Abtract: 
        Electrocardiogram (ECG) is a widely used reliable, non-invasive approach for cardiovascular disease diagnosis. With the rapid growth of ECG examinations and the insufficiency of cardiologists, accurate and automatic diagnosis of ECG signals has become a hot research topic. Deep learning methods have demonstrated promising results in predictive healthcare tasks. In this paper, we developed a deep neural network for multi-label classification of cardiac arrhythmias in 12-lead ECG recordings. Experiments on a public 12-lead ECG dataset showed the effectiveness of our method. The proposed model achieved an average area under the receiver operating characteristic curve (AUC) of 0.970 and an average F1 score of 0.813. The deep model showed superior performance than 4 machine learning methods learned from extracted expert features. Besides, the deep models trained on single-lead ECGs produce lower performance than using all 12 leads simultaneously. The best-performing leads 

In [217]:
links=soup.find_all('a',{'href':True})
links

[<a class="is-sr-only" href="#main-container">Skip to main content</a>,
 <a class="level-item" href="https://cornell.edu/"><img alt="Cornell University" aria-label="logo" src="https://static.arxiv.org/static/base/0.16.8/images/cornell-reduced-white-SMALL.svg" width="200"/></a>,
 <a href="https://confluence.cornell.edu/x/ALlRF">We gratefully acknowledge support from<br/> the Simons Foundation and member institutions.</a>,
 <a aria-label="arxiv-logo" class="arxiv" href="https://arxiv.org/"><img alt="arXiv" aria-label="logo" src="https://static.arxiv.org/static/base/0.16.8/images/arxiv-logo-web.svg" width="85"/></a>,
 <a href="https://arxiv.org/help">Help</a>,
 <a href="https://arxiv.org/search/advanced">Advanced Search</a>,
 <a href="https://arxiv.org/login">Login</a>,
 <a href="https://github.com/arXiv/arxiv-search/releases">Search v0.5.6 released 2020-02-24</a>,
 <a href="/search/advanced?terms-0-term=healthcare&amp;terms-0-field=all&amp;size=50&amp;order=-announced_date_first">Advance

In [218]:
import re

In [219]:
final_title =[]
Arxiv_id =[]
Article_link =[]
pdf_link=[]
Abstract =[]
Tags=[]
Authors=[]
Submitted_date=[]
pages = browser.find_element_by_class_name("pagination-next")
try:
    while True:
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        li=soup.find_all('li',{'class':'arxiv-result'})

        for s in li:
            #Extract Title
            title=s.find('p' , class_="title is-5 mathjax")
            clean_title=title.text.strip()
            final_title.append(clean_title)
            #Extract Arxiv ids 
            item=s.find('p' ,class_='list-title is-inline-block')
            clean_ids=item.text
            ids=re.findall("20.*", clean_ids)
            Arxiv_id.append(ids)
            #
            #Extract Article link
            link=s.find_all("a", href=lambda href: href and href.startswith("https://arxiv.org/abs/"))
            clean_link=str(link)
            final_Article_link = re.findall(r'"([^"]*)"',clean_link)
            Article_link.append(final_Article_link ) 
            
            #Extract pdf_link
            link=s.find_all("a", href=lambda href: href and href.startswith("https://arxiv.org/pdf/"))
            clean_link=str(link)
            final_pdf_link = re.findall(r'"([^"]*)"',clean_link)
            pdf_link.append(final_pdf_link) 
            
            #Extract Abstract
            link=s.find('span' , class_ = "abstract-full has-text-grey-dark mathjax")
            text=link.text.strip()
            clean_text= re.sub("▽ More", " ", text)
            Abstract.append(clean_text)   
            #Extract Tags
            tag =s.find('div' , class_="tags is-inline-block")
            clean_tags=tag.text.split()
            tags = ",".join(clean_tags)
            Tags.append(tags)
            
            #Extract Authors
            link = soup.find('p',class_="authors")
            clean_text=link.text.replace("\n", '')
            Authors.append(clean_text)
            
            #Extract dates
            dates=s.find('p',class_='is-size-7')
            clean_dates = re.findall("\d+\s+\w+,\s+\d{4}",dates.text)
            Submitted_date.append(clean_dates)
        pages = browser.find_element_by_class_name("pagination-next")
        pages.click()
        
except Exception as Ex:
    print(Ex)

Message: Element <a class="pagination-next is-invisible" href=""> could not be scrolled into view



In [220]:
df = pd.DataFrame (final_title,columns=['Title'])
df['Arxiv_id']=Arxiv_id
df['Article_link']=Article_link
df['pdf_link']=pdf_link
df['Abstract']=Abstract
df['Tags']=Tags
df['Authors']=Authors
df['Submitted_date']=Submitted_date

In [221]:
df.head()

,Title,Arxiv_id,Article_link,pdf_link,Abstract,Tags,Authors,Submitted_date
0,Interpretable Deep Learning for Automatic Diag...,[2010.10328],[https://arxiv.org/abs/2010.10328],[https://arxiv.org/pdf/2010.10328],Electrocardiogram (ECG) is a widely used relia...,"cs.LG,eess.SP","Authors:Dongdong Zhang, Xiaohui Yu...","[20 October, 2020]"
1,Leveraging Technology for Healthcare and Retai...,[2010.10285],[https://arxiv.org/abs/2010.10285],[https://arxiv.org/pdf/2010.10285],Health data is a sensitive category of persona...,cs.CR,"Authors:Dongdong Zhang, Xiaohui Yu...","[20 October, 2020]"
2,Towards new forms of particle sensing and mani...,[2010.10269],[https://arxiv.org/abs/2010.10269],[https://arxiv.org/pdf/2010.10269],Close to half of the world population have sma...,"eess.IV,eess.SP,physics.med-ph","Authors:Dongdong Zhang, Xiaohui Yu...","[8 September, 2020]"
3,University Operations During a Pandemic: A Fle...,[2010.10112],[https://arxiv.org/abs/2010.10112],[https://arxiv.org/pdf/2010.10112],Modeling infection spread during pandemics is ...,eess.SY,"Authors:Dongdong Zhang, Xiaohui Yu...","[20 October, 2020]"
4,The Role of Robotics in Infectious Disease Crises,[2010.09909],[https://arxiv.org/abs/2010.09909],[https://arxiv.org/pdf/2010.09909],The recent coronavirus pandemic has highlighte...,"cs.RO,cs.CY","Authors:Dongdong Zhang, Xiaohui Yu...","[19 October, 2020]"


In [222]:
df.shape

(1890, 8)

In [223]:
# https://stackoverflow.com/a/47091490/4084039
import re
def decontracted(phrase): # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
# general
    phrase = re.sub(r"n\'t", " not", phrase) 
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase) 
    phrase = re.sub(r"\'d", " would", phrase) 
    phrase = re.sub(r"\'ll", " will", phrase) 
    phrase = re.sub(r"\'t", " not", phrase) 
    phrase = re.sub(r"\'ve", " have", phrase) 
    phrase = re.sub(r"\'m", " am", phrase) 
    return phrase

In [224]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not' stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]      

In [225]:
#clean titles
from tqdm import tqdm
preprocessed_titles = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_titles.append(sent.lower().strip())

100%|██████████| 1890/1890 [00:00<00:00, 10096.96it/s]


In [226]:
#clean abstract
from tqdm import tqdm
preprocessed_Abstract = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Abstract'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_Abstract.append(sent.lower().strip())

100%|██████████| 1890/1890 [00:01<00:00, 1609.83it/s]


In [227]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer


In [228]:
df['preprocessed_titles']=preprocessed_titles
df['preprocessed_Abstract']=preprocessed_Abstract

In [229]:
count_vectorizer = CountVectorizer() 
vsm=count_vectorizer.fit_transform(df['preprocessed_Abstract'].values)


In [230]:
import pandas as pd
pd.DataFrame(vsm.todense(), columns=count_vectorizer.get_feature_names()).head()

,00,000,001,0010,003,004,0045,005,0054,006,...,zhang,zigbee,zika,zin,zinc,zip,zkp,zone,zones,zteac
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [255]:
query=['natural language processing']

In [256]:
## Vectorize the query
query_vector=count_vectorizer.transform(query)

In [257]:
from sklearn.metrics.pairwise import cosine_similarity

In [258]:
similarity_vals=cosine_similarity(query_vector, vsm)[0]

In [259]:
similarity_vals

array([0.        , 0.        , 0.        , ..., 0.03126527, 0.        ,
       0.1723455 ])

In [260]:
import numpy as np
rankings_bow=np.argsort(similarity_vals)[::-1]

In [261]:
rankings_bow

array([ 819,  770,   11, ..., 1236, 1237,    0])

In [262]:
countvector_results=[preprocessed_Abstract[i] for i in rankings_bow]

# Apply TFIDF on Preprocessed_Abstract

In [263]:
tfidf_vectorizer = TfidfVectorizer()
tfidf=tfidf_vectorizer.fit_transform(df['preprocessed_Abstract'].values) # fit has to happen only on train data
# we use the fitted CountVectorizer to convert the text to vector
tfidf

<1890x14990 sparse matrix of type '<class 'numpy.float64'>'
	with 176894 stored elements in Compressed Sparse Row format>

In [264]:
import pandas as pd
pd.DataFrame(tfidf.todense(), columns=tfidf_vectorizer.get_feature_names()).head()

,00,000,001,0010,003,004,0045,005,0054,006,...,zhang,zigbee,zika,zin,zinc,zip,zkp,zone,zones,zteac
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
query=['natural language processing']

In [266]:
## Vectorize the query
query_vector=tfidf_vectorizer.transform(query)

In [267]:
query_vector

<1x14990 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [268]:
from sklearn.metrics.pairwise import cosine_similarity

In [269]:
similarity_vals=cosine_similarity(query_vector,tfidf)[0]

In [270]:
similarity_vals

array([0.        , 0.        , 0.        , ..., 0.02193747, 0.        ,
       0.17026599])

In [271]:
import numpy as np
rankings_tfidf=np.argsort(similarity_vals)[::-1]
rankings_tfidf

array([ 819,  846,  770, ..., 1236, 1237,    0])

In [272]:
tfidf_result=[preprocessed_Abstract[i] for i in rankings_tfidf]

In [273]:
countvector_results[:5]

['a natural language interface nli facilitates users pose queries retrieve information database without using artificial language structured query language sql several applications various domains including healthcare customer support search engines require elaborating structured data information text moreover many issues explored including configuration complexity processing intensive algorithms popularity relational databases due translating natural language database query become secondary area investigation the emerging trend querying systems speech enabled interfaces revived natural language database queries research area the last survey published topic six years ago 2013 to best knowledge no recent study found discusses current state art translations frameworks natural language structured non structured query languages in paper reviewed 47 frameworks 2008 2018 out 47 35 closely relevant work sql based frameworks categorized statistical symbolic connectionist approaches whereas nos

In [274]:
tfidf_result[:5]

['a natural language interface nli facilitates users pose queries retrieve information database without using artificial language structured query language sql several applications various domains including healthcare customer support search engines require elaborating structured data information text moreover many issues explored including configuration complexity processing intensive algorithms popularity relational databases due translating natural language database query become secondary area investigation the emerging trend querying systems speech enabled interfaces revived natural language database queries research area the last survey published topic six years ago 2013 to best knowledge no recent study found discusses current state art translations frameworks natural language structured non structured query languages in paper reviewed 47 frameworks 2008 2018 out 47 35 closely relevant work sql based frameworks categorized statistical symbolic connectionist approaches whereas nos

result tf_idf works better than bow